In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import time
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import hog
import os
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r /content/gdrive/MyDrive/Faces_Data/* /content

In [ ]:
test_dir="/content/val"
train_dir="/content/train"
train_dir_0 = train_dir + '/0'
train_dir_1 = train_dir + '/1'
test_dir_0 = test_dir + '/0'
test_dir_1 = test_dir + '/1'

In [ ]:
print('number of 0 training images - ',len(os.listdir(train_dir_0)))
print('number of 1 training images - ',len(os.listdir(train_dir_1)))
print('number of 0 testing images - ',len(os.listdir(test_dir_0)))
print('number of 1 testing images - ',len(os.listdir(test_dir_1)))

number of 0 training images -  57
number of 1 training images -  264
number of 0 testing images -  19
number of 1 testing images -  82


In [ ]:
[0] * 3 + [1]*4

[0, 0, 0, 1, 1, 1, 1]

In [ ]:
#Labels
tr_labels = [1] * len(os.listdir(train_dir_1)) + [0] * len(os.listdir(train_dir_0))
ts_labels = [1] * len(os.listdir(test_dir_1)) + [0] * len(os.listdir(test_dir_0))

In [ ]:
#tr-data
tr_data = []
for filename in os.listdir(train_dir_1):
        image = cv2.imread(os.path.join(train_dir_1, filename))
        tr_data.append(image)
for filename in os.listdir(train_dir_0):
        image = cv2.imread(os.path.join(train_dir_0, filename))
        tr_data.append(image)

In [ ]:
#ts-data
ts_data = []
for filename in os.listdir(test_dir_1):
        image = cv2.imread(os.path.join(test_dir_1, filename))
        ts_data.append(image)
for filename in os.listdir(test_dir_0):
        image = cv2.imread(os.path.join(test_dir_0, filename))
        ts_data.append(image)

>  Converting to grayscale images

In [ ]:
tr_data_gray = [cv2.cvtColor(tr_data[i] , cv2.COLOR_BGR2GRAY) for i in range(len(tr_data))]
ts_data_gray = [cv2.cvtColor(ts_data[i] , cv2.COLOR_BGR2GRAY) for i in range(len(ts_data))]

In [ ]:
tr_labels = np.array(tr_labels).reshape(len(tr_labels),1)
ts_labels = np.array(ts_labels).reshape(len(ts_labels),1)

>  ## HOG Descriptor

Let’s look at some important aspects of HOG that makes it different from other feature descriptors:

* The HOG descriptor focuses on the structure or the shape of an object. Now you might ask, how is this different from the edge features we extract for images? In the case of edge features, we only identify if the pixel is an edge or not. **HOG is able to provide the edge direction as well.** This is done by extracting the gradient and orientation (or you can say magnitude and direction) of the edges
* Additionally, these orientations are calculated in **‘localized’** portions. This means that the complete image is broken down into smaller regions and for each region, the gradients and orientation are calculated. We will discuss this in much more detail in the upcoming sections
* Finally the HOG would generate a **Histogram** for each of these regions separately. The histograms are created using the gradients and orientations of the pixel values, hence the name ‘Histogram of Oriented Gradients’ 

SOURCE: <https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/>

Using HOG function from skimage.
* if you set the parameter ‘visualize = True’, it will return an image of the HOG.

+++++++ ======================================================= ++++++++

Before going ahead, let me give you a basic idea of what each of these hyperparameters represents.
* The orientations are the number of buckets we want to create. Since I want to have a 9 x 1 matrix, I will set the orientations to 9

* pixels_per_cell defines the size of the cell for which we create the histograms.We used 8 x 8 cells and you can choose to change this value.

* We have another hyperparameter cells_per_block which is the size of the block over which we normalize the histogram. Here, we mention the cells per blocks and not the number of pixels. So, instead of writing 32 x 32, we will use 2 x 2 here.

In [ ]:
ppc =8
cb=4

In [ ]:
#tr-data
tr_hog_features=[]
tr_hog_image=[]
for image in tqdm(tr_data_gray):
    fd , hogim = hog(image , orientations=9 , pixels_per_cell=(ppc , ppc) , block_norm='L2' , cells_per_block=(cb,cb) , visualize=True )
    tr_hog_image.append(hogim)
    tr_hog_features.append(fd)

100%|██████████| 321/321 [00:48<00:00,  6.61it/s]


In [ ]:
#ts-data
ts_hog_features=[]
ts_hog_image=[]
for image in tqdm(ts_data_gray):
    fd , hogim = hog(image , orientations=9 , pixels_per_cell=(ppc , ppc) , block_norm='L2' , cells_per_block=(cb,cb) , visualize=True) 
    ts_hog_image.append(hogim)
    ts_hog_features.append(fd)

100%|██████████| 101/101 [00:15<00:00,  6.61it/s]


In [ ]:
#> VISUALISING HOG IMAGES
'''
fig = plt.figure(figsize=(20,15))

for i in range(1,10):
    index = random.randint(1,10770)
    plt.subplot(3,3,i)
    plt.imshow(hog_image[index])
    plt.xlabel(Labels[index])
plt.show()
'''

'\nfig = plt.figure(figsize=(20,15))\n\nfor i in range(1,10):\n    index = random.randint(1,10770)\n    plt.subplot(3,3,i)\n    plt.imshow(hog_image[index])\n    plt.xlabel(Labels[index])\nplt.show()\n'

>  ## Create SVM model to fit

In [ ]:
#train
tr_hog_features = np.array(tr_hog_features)
tr_df = np.hstack((tr_hog_features,tr_labels))
#test
ts_hog_features = np.array(ts_hog_features)
ts_df = np.hstack((ts_hog_features,ts_labels))

>  Using PCA for dimension reduction

In [ ]:
from sklearn.decomposition import PCA
t= time.time()
pca = PCA(n_components=150 , svd_solver='randomized' , whiten=True).fit(tr_hog_features)
print("Time evolved", time.time()-t)

Time evolved 7.319055795669556


In [ ]:
joblib.dump(pca, 'pca_model.pkl')

['pca_model.pkl']

In [ ]:
print("Projecting the input data on the orthonormal basis")
t0 = time.time()
tr_hog_features_pca = pca.transform(tr_hog_features)
ts_hog_features_pca = pca.transform(ts_hog_features)
print("done in %0.3fs" % (time.time() - t0))

Projecting the input data on the orthonormal basis
done in 0.441s


In [ ]:
svm = SVC(kernel='rbf' , class_weight='balanced' , C=1000 , gamma=0.0082)
svm.fit(tr_hog_features,tr_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1000, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.0082,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [ ]:
print(svm.score(ts_hog_features,ts_labels))

0.8613861386138614


In [ ]:
#Save the Model

In [ ]:
# Save the model as a pickle in a file
joblib.dump(svm, 'svm_model.pkl')
 
# Load the model from the file
#knn_from_joblib = joblib.load('filename.pkl') 
# Use the loaded model to make predictions
#knn_from_joblib.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['svm_model.pkl']

In [ ]:
np.__version__

'1.19.5'

In [ ]:
import sklearn
sklearn.__version__

'0.22.2.post1'